In [ ]:
%pylab inline
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [ ]:
def get_data(d):
    x = np.load(f"{d}/x.npz")['arr_0']
    y = np.load(f"{d}/y.npz")['arr_0']
    
    return x,y

x, y= get_data("ml")


# maybe transform data

def transform(y, tht=45):
    return np.arcsinh(y*tht)/tht

In [ ]:
from scipy.stats import probplot


yr = np.random.choice(y.ravel(), 10000, replace=False)
probplot(transform(yr, 45), plot=plt);

45 seems like a reasonable value for the transform

In [ ]:
n = x.shape[0]
percent_train = .6
ntrain = int(percent_train * n)

train_idx = np.arange(ntrain)
test_idx = np.arange(ntrain, n)

x_train, y_train = x[train_idx], y[train_idx]
x_test, y_test = x[test_idx], y[test_idx]

MCA prediction

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
def score_mod_xform(mod):
    tht = 10
    z_train = np.arcsinh(tht*y_train)/tht
    zt = mod.fit(x_train, z_train).predict(x_test)
    yt = np.sinh(zt*tht)/tht
    return r2_score(y_test, yt, multioutput="variance_weighted")

def score_mod(mod):
    return mod.fit(x_train, y_train).score(x_test, y_test)


def plot_coef(mod):
    I = np.eye(x.shape[1])
    coef= mod.predict(I)
    plt.pcolormesh(coef)
    plt.colorbar()

def summarize_performance(mod):
    r2_xform = score_mod_xform(mod)
    r2 = score_mod(mod)
    plot_coef(mod)
    
    print(f"R2 = {r2}; arcsinh r2 = {r2_xform}")
    
    

In [ ]:
from mca import MCA

In [ ]:
mca = make_pipeline(MCA(60), LinearRegression(fit_intercept=False))
summarize_performance(mca)

In [ ]:
mca = make_pipeline(MCA(10), LinearRegression(fit_intercept=False))
summarize_performance(mca)

Ridge regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
summarize_performance(Ridge(1000))

Partial least squares

In [ ]:
from sklearn.cross_decomposition import PLSRegression

summarize_performance(PLSRegression(10, scale=False))

Principal component regression

In [ ]:
pipe = make_pipeline(TruncatedSVD(40), LinearRegression())
summarize_performance(pipe)

From this it seems that the maximum covariance analysis returns the most accurate results it only does so when an extreme number of modes is used, but to my eye the linear response function obtained by ridge regression is the most plausible since it smooth. The other methdos all have visible oscillatory phenomena. Maybe partial least squares will work better if run in a different mode, but it is the most computationally expensive algorithm of the three. 

Also, taking the arcsinh transform does not appear to help either. It seems the extreme convection events cannot be captured well by generalized linear models.